In [78]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [79]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_teddynote.messages import AgentStreamParser

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_community.agent_toolkits import FileManagementToolkit
import os

In [80]:
@tool
def latest_news(k: int = 5) -> List[Dict[str, str]]:        # 최신 뉴스를 검색하는 도구
    """Look up latest news"""

    print(f'최신 뉴스 검색')

    news_tool = GoogleNews()

    return news_tool.search_latest(k=k)                     # 최신 뉴스 k개를 검색해서 리턴

In [81]:
if not os.path.exists('tmp2'):                              # 작업 폴더 생성
    os.mkdir('tmp2')

working_directory = 'tmp2'                                   # 작업 디렉토리 설정 

tools = FileManagementToolkit(                              # 파일 관리 도구를 가져온다
    root_dir=str(working_directory)
).get_tools()


tools.append(latest_news)                                   # latest_news 도구를 tools에 추가한다

In [82]:
for tool in tools:                                          # 도구 목록
    print(f' - {tool.name} : {tool.description}')

 - copy_file : Create a copy of a file in a specified location
 - file_delete : Delete a file
 - file_search : Recursively search for files in a subdirectory that match the regex pattern
 - move_file : Move or rename a file from one location to another
 - read_file : Read file from disk
 - write_file : Write file to disk
 - list_directory : List files and directories in a specified folder
 - latest_news : Look up latest news


In [83]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

In [84]:
prompt = ChatPromptTemplate.from_messages(                  # 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. 도구의 역할과 이름을 입력을 입력합니다.
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `latest_news` tool to find latest news. "         
            "Make sure to use the `file_management` tool to manage files. ",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [85]:
agent = create_tool_calling_agent(llm, tools, prompt)       # agent 생성

In [86]:
agent_executor = AgentExecutor(                             # AgentExecutor 생성 
    agent=agent,                                            # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                                            # agent 가 사용할 수 있는 도구 목록
    verbose=True,
    max_iterations=10,                                      # 최대 10번 까지 반복
    max_execution_time=10,                                  # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

In [87]:
store = {}                              # 채팅 기록을 위한 메모리

def get_session_history(session_ids):   # session_id 를 기반으로 세션 기록을 가져오는 함수    
    if session_ids not in store:        # session_id 가 store에 없는 경우        
        store[session_ids] = ChatMessageHistory()   # 새로운 ChatMessageHistory 객체를 딕셔너리에 저장
    
    return store[session_ids]           # 해당 세션 ID에 대한 세션 기록 리턴


agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,                                    # session_id 를 기반으로 세션 기록을 가져오는 함수
    input_messages_key='input',                             # 프롬프트의 질문으로 들어가는 key: input
    history_messages_key='chat_history'                     # 프롬프트의 대화 기록이 들어가는 key: chat_history
)

In [88]:
result = agent_with_chat_history.stream(
    {
        'input': '최신 뉴스 5개를 검색하고, 각 뉴스의 제목을 파일명으로 하는 파일명을 생성하고(.txt), '
        '파일의 뉴스 내용은 뉴스의 내용과 url을 추가하세요.'
    }, 
    config={'configurable': {'session_id': 'abc123'}}            
)

In [89]:
for step in result:

    if 'actions' in step:
        print('==' * 50)
        print(f'[actions]')
        # print(step['actions'])
        print('[도구 호출]')
        print(f"Tool : {step['actions'][0].tool}")
        print(f"query : {step['actions'][0].tool_input}")
        print(f"Log: ", end='')
        print(f"{step['actions'][0].log}")
        # print(f"message log: {step['actions'][0].message_log}")
        print('==' * 50)

    elif 'steps' in step:
        print('\n')
        print('==' * 50)
        print(f'[steps]')
        # print(step['steps'])
        print(f"검색어: {step['steps'][0].action.tool_input}")
        # print(f"message log: {step['steps'][0].action.message_log}")
        print(f"[관찰 내용]")
        print(f"Observation:")
        print(f"{step['steps'][0].observation}")
        print('==' * 50)

    elif 'output' in step:
        print('==' * 50)
        # print(f"[output]: {step['output']}")
        print(f"[output]")
        print(f"step['messages']:")
        print(f"{step['messages'][0].content}")
        print('==' * 50)



> Entering new None chain...
[actions]
[도구 호출]
Tool : latest_news
query : {'k': 5}
Log: 
Invoking: `latest_news` with `{'k': 5}`




Invoking: `latest_news` with `{'k': 5}`


최신 뉴스 검색
[{'url': 'https://news.google.com/rss/articles/CBMihgFBVV95cUxQWkp2Z3phZmh2Ty1Uc2ZXQ2JzT1hJUzJxdC1GWmM4RkVUb2g3RVdPbFJualM3Z04zd05jbVhfenMxb056ZFZ3ZHJ6MnMtNDhSaFVtS3NQUEFOcUt2Mk1MZ3M3RmNmZGhZXzNXMFJJOGFZdHY0czlSbjZYRjZuakR6SzY1SHR1QdIBmgFBVV95cUxQTlNyaXhWSExqMjJoTnl3dkliV3RDYVpDTnphdnRRYnF0dW1Gc0h4LTY0emZ5dFBRLVhTeU43ckVsVDlvYllpUUYxUFNMRFFjdHl3Vk1RWFpwQ0kxZlpmbFF3QUdoa0xCUDBLcVoyTmxrc21iQXhwZ2FXbzFGVmIzVWRLcHpDVFlCNzJ3VkQ3S3hkaUNHS0Mxekh3?oc=5', 'content': '하늘이 살해 교사, ‘흉기’ ‘살인사건 기사’ 검색…경찰 “계획범행 무게” - 조선일보'}, {'url': 'https://news.google.com/rss/articles/CBMickFVX3lxTE55UUlxa0N1OGp4c2JxYUlOaWZHbVNzaUtNcDRNVUJEUEhPNGFVdWo1SzNUcWRoVEFZT3NKbkU1eTBqVzItVmFKM0RFdEM4aFNTRUdITzdVdTVQTGZmRnRHSnR2SkF5dUdIVTk0TGdhemtYUQ?oc=5', 'content': '[속보] 윤 대통령 쪽 “대통령, 내일 구속취소 청구 심문 참여” - 한겨레'}, {'url': 'https://news.google.

Stopping agent prematurely due to triggering stop condition


[actions]
[도구 호출]
Tool : write_file
query : {'file_path': '하늘이 살해 교사, ‘흉기’ ‘살인사건 기사’ 검색…경찰 “계획범행 무게”.txt', 'text': '하늘이 살해 교사, ‘흉기’ ‘살인사건 기사’ 검색…경찰 “계획범행 무게” - 조선일보\nURL: https://news.google.com/rss/articles/CBMihgFBVV95cUxQWkp2Z3phZmh2Ty1Uc2ZXQ2JzT1hJUzJxdC1GWmM4RkVUb2g3RVdPbFJualM3Z04zd05jbVhfenMxb056ZFZ3ZHJ6MnMtNDhSaFVtS3NQUEFOcUt2Mk1MZ3M3RmNmZGhZXzNXMFJJOGFZdHY0czlSbjZYRjZuakR6SzY1SHR1QdIBmgFBVV95cUxQTlNyaXhWSExqMjJoTnl3dkliV3RDYVpDTnphdnRRYnF0dW1Gc0h4LTY0emZ5dFBRLVhTeU43ckVsVDlvYllpUUYxUFNMRFFjdHl3Vk1RWFpwQ0kxZlpmbFF3QUdoa0xCUDBLcVoyTmxrc21iQXhwZ2FXbzFGVmIzVWRLcHpDVFlCNzJ3VkQ3S3hkaUNHS0Mxekh3?oc=5'}
Log: 
Invoking: `write_file` with `{'file_path': '하늘이 살해 교사, ‘흉기’ ‘살인사건 기사’ 검색…경찰 “계획범행 무게”.txt', 'text': '하늘이 살해 교사, ‘흉기’ ‘살인사건 기사’ 검색…경찰 “계획범행 무게” - 조선일보\nURL: https://news.google.com/rss/articles/CBMihgFBVV95cUxQWkp2Z3phZmh2Ty1Uc2ZXQ2JzT1hJUzJxdC1GWmM4RkVUb2g3RVdPbFJualM3Z04zd05jbVhfenMxb056ZFZ3ZHJ6MnMtNDhSaFVtS3NQUEFOcUt2Mk1MZ3M3RmNmZGhZXzNXMFJJOGFZdHY0czlSbjZYRjZuakR6SzY1SHR1QdIBm

In [90]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool[0]}")
    print(f'===== 도구 호출 =====')

In [91]:
def obervation_callback(observation) -> None:
    print(f'\n===== 관찰 내용 =====')
    print(f"Observation: {observation[0].observation}")
    print(f'===== 관찰 내용 =====')

In [92]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(f"{result}")
    print(f'===== 최종 답변 =====')

In [93]:
result = agent_with_chat_history.stream(
    {
        'input': '이전에 생성한 파일 제목 맨 앞에 제목에 어울리는 emoji를 추가하여 파일명을 변경하세요. '
        '파일도 깔끔하게 변경하세요.'
    }, 
    config={'configurable': {'session_id': 'abc123'}}            
)

In [94]:
for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)  



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : tool='list_directory' tool_input={} log='\nInvoking: `list_directory` with `{}`\n\n\n' message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ln4oOd13EsyDHOg0CPteBjTQ', 'function': {'arguments': '{}', 'name': 'list_directory'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1'}, id='run-b1b0d073-44c7-46fa-a4ee-7d335f684bf5', tool_calls=[{'name': 'list_directory', 'args': {}, 'id': 'call_ln4oOd13EsyDHOg0CPteBjTQ', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'list_directory', 'args': '{}', 'id': 'call_ln4oOd13EsyDHOg0CPteBjTQ', 'index': 0, 'type': 'tool_call_chunk'}])] tool_call_id='call_ln4oOd13EsyDHOg0CPteBjTQ'
===== 도구 호출 =====

Invoking: `list_directory` with `{}`


[속보] 윤 대통령 쪽 “대통령, 내일 구속취소 청구 심문 참여”.txt
“내가 김용현한테 ‘챙기라’고”…민주, 명태균 경호처 인사개입 정황 공개.txt
하늘이 살해 교사, ‘흉기’ ‘살인사건 기사’

In [95]:
result = agent_with_chat_history.stream(
    {
        'input': '이전에 생성한 모든 파일을 news 폴더를 생성한 뒤 해당 폴더에 모든 파일을 복사하세요. '
        '내용도 동일하게 복사하세요.'
    }, 
    config={'configurable': {'session_id': 'abc123'}}            
)

for step in result:

    if 'actions' in step:
        print('==' * 50)
        print(f'[actions]')
        # print(step['actions'])
        print('[도구 호출]')
        print(f"Tool : {step['actions'][0].tool}")
        print(f"query : {step['actions'][0].tool_input}")
        print(f"Log: ", end='')
        print(f"{step['actions'][0].log}")
        # print(f"message log: {step['actions'][0].message_log}")
        print('==' * 50)

    elif 'steps' in step:
        print('\n')
        print('==' * 50)
        print(f'[steps]')
        # print(step['steps'])
        print(f"검색어: {step['steps'][0].action.tool_input}")
        # print(f"message log: {step['steps'][0].action.message_log}")
        print(f"[관찰 내용]")
        print(f"Observation:")
        print(f"{step['steps'][0].observation}")
        print('==' * 50)

    elif 'output' in step:
        print('==' * 50)
        # print(f"[output]: {step['output']}")
        print(f"[output]")
        print(f"step['messages']:")
        print(f"{step['messages'][0].content}")
        print('==' * 50)



> Entering new None chain...
[actions]
[도구 호출]
Tool : list_directory
query : {}
Log: 
Invoking: `list_directory` with `{}`




Invoking: `list_directory` with `{}`


[속보] 윤 대통령 쪽 “대통령, 내일 구속취소 청구 심문 참여”.txt
⚖️ 헌재, ‘한덕수 총리 탄핵 정족수’ 권한쟁의 심판 70분 만에 종결...선고일 추후 결정.txt
📢 “내가 김용현한테 ‘챙기라’고”…민주, 명태균 경호처 인사개입 정황 공개.txt
🔪 하늘이 살해 교사, ‘흉기’ ‘살인사건 기사’ 검색…경찰 “계획범행 무게”.txt

[steps]
검색어: {}
[관찰 내용]
Observation:
[속보] 윤 대통령 쪽 “대통령, 내일 구속취소 청구 심문 참여”.txt
⚖️ 헌재, ‘한덕수 총리 탄핵 정족수’ 권한쟁의 심판 70분 만에 종결...선고일 추후 결정.txt
📢 “내가 김용현한테 ‘챙기라’고”…민주, 명태균 경호처 인사개입 정황 공개.txt
🔪 하늘이 살해 교사, ‘흉기’ ‘살인사건 기사’ 검색…경찰 “계획범행 무게”.txt
[actions]
[도구 호출]
Tool : copy_file
query : {'source_path': '[속보] 윤 대통령 쪽 “대통령, 내일 구속취소 청구 심문 참여”.txt', 'destination_path': 'news/[속보] 윤 대통령 쪽 “대통령, 내일 구속취소 청구 심문 참여”.txt'}
Log: 
Invoking: `copy_file` with `{'source_path': '[속보] 윤 대통령 쪽 “대통령, 내일 구속취소 청구 심문 참여”.txt', 'destination_path': 'news/[속보] 윤 대통령 쪽 “대통령, 내일 구속취소 청구 심문 참여”.txt'}`




Invoking: `copy_file` with `{'source_path': '[속보] 윤 대통령 쪽 “대통령, 내일 구